Лабораторна робота № 6
Смислов Даніл
ІП-01

Спочатку імпортуємо потрібні для роботи бібліотеки

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np


Далі імпортуємо дані з файлу titanic.csv з кодуванням 'cp1252' та розділювачем ','. Розглянемо структуру датафрейму.

In [ ]:
def getDataset():
    data = pd.read_csv('titanic.csv',sep=',',encoding='cp1252',decimal='.')
    return data 

dataset = getDataset()
dataset.info()
dataset.head(10)

Давайте змінимо тип стовпчику 'Pclass'

In [ ]:
dataset['Pclass'] = dataset['Pclass'].astype(str)
dataset.info()

Тепер давайте видалимо стовпчики, які нам не будуть потрібні для побудови моделі.

In [ ]:
dataset = dataset.drop(columns=['PassengerId','Name','Ticket'])

dataset.head(2)

Давайте перевіримо дані на пропуски.

In [ ]:
import matplotlib.patches as mpatches
import seaborn as sbrn
plt.figure(figsize =(12,12))
plt.title('Heat map of missing data')
lst = []
is_patch =mpatches.Patch(color = 'black', label = 'Data')
non_patch =mpatches.Patch(color = 'white', label = 'Missing data')
lst.append(is_patch)
lst.append(non_patch)
plt.legend(handles = lst,bbox_to_anchor = (1,1))
sbrn.heatmap(dataset.isna(),cmap = sbrn.color_palette(['Black','White']))
plt.show()


Тепер давайте ще подивимось на к-сть пропущених даних.

In [ ]:
number = dataset.isnull().sum().sort_values(ascending=False)
percent = (number / (dataset.count()+dataset.isnull().sum())*100).sort_values(ascending=False)
missingData = pd.concat([number,percent],axis=1,keys = ['Number NaN','Percent NaN'])
missingData.head(10)

Як ми бачимо, стовпець 'Cabin' має досить багато пропущених значень, тому давайте видалимо його. В стовцях 'Age' та 'Embarked' замінимо пропущені значення моди.

In [ ]:
dataset = dataset.drop(columns = 'Cabin')
dataset['Embarked'] = dataset['Embarked'].fillna(dataset['Embarked'].mode()[0])
dataset['Age'] = dataset['Age'].fillna(dataset['Age'].mode()[0])
number = dataset.isnull().sum().sort_values(ascending=False)
percent = (number / (dataset.count()+dataset.isnull().sum())*100).sort_values(ascending=False)
missingData = pd.concat([number,percent],axis=1,keys = ['Number NaN','Percent NaN'])
missingData.head(10)

Як ми бачимо, тепер всі дані заповненні, тож ми готові працювати з даними. Тепер давайте розподілимо дані на тренувальні та дані для тесту.

In [ ]:
from sklearn.model_selection import train_test_split
trainingData, testingData  = train_test_split(dataset,test_size=0.2,random_state = 1)
features = pd.concat([trainingData,testingData]).reset_index(drop = True)
features = pd.get_dummies(features)
trainingData = features.iloc[:trainingData.shape[0],:]
testingData = features.iloc[trainingData.shape[0]:,:]


Тепер визначимо змінні X та Y для тренувальних та тестувальних даних.

In [ ]:
trainingX  = trainingData.drop(columns='Survived')
trainingY = trainingData['Survived']

testingX  = testingData.drop(columns='Survived')
testingY = testingData['Survived']

print(trainingY)

Додамо необхідні бібліотеки для побудови моделей, побудуємо їх та перевіримо на тестових даних. Спочатку класифікатор Decision Tree.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
decisionTree = DecisionTreeClassifier(max_depth=3,random_state=1)
tree_scores = cross_val_score(decisionTree,trainingX,trainingY,cv = 5)
print('Cvs in Decision Tree:', tree_scores.mean())
decisionTree.fit(trainingX,trainingY)
print('Score of Decision Tree:',decisionTree.score(testingX,testingY)) 

Тепер класифікатор Random Forest. 

In [ ]:
randomForest = RandomForestClassifier(max_depth=5)
tree_scores = cross_val_score(randomForest,trainingX,trainingY,cv = 5)
print('Cvs in Random Forest:', tree_scores.mean())
randomForest.fit(trainingX,trainingY)
print('Score of Random Forest:',randomForest.score(testingX,testingY)) 

Тепер класифікатор Gradient Boosting.

In [ ]:
gradientBoosting = GradientBoostingClassifier(learning_rate = 0.1)
tree_scores = cross_val_score(gradientBoosting,trainingX,trainingY,cv = 5)
print('Cvs in Gradient Boosting:', tree_scores.mean())
gradientBoosting.fit(trainingX,trainingY)
GradientBoostingClassifier(learning_rate = 0.1)
print('Score of Gradient Boosting:',gradientBoosting.score(testingX,testingY)) 

Тепер класифікатор AdaBoost.

In [ ]:
adaBoosting = AdaBoostClassifier(learning_rate = 0.3)
tree_scores = cross_val_score(adaBoosting,trainingX,trainingY,cv = 5)
print('Cvs in Ada Boosting:', tree_scores.mean())
adaBoosting.fit(trainingX,trainingY)
print('Score of Ada Boosting:',adaBoosting.score(testingX,testingY)) 

Як ми бачимо, на тестових даних найкраще себе показав класифікатор Decision Tree, адже в нього найвище значення score. Але можна сказати, що моделі в цілому показали досить схожий результат.